## Music genre classification using spectograms

In [ ]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from google.colab import drive



# Mount Google Drive to access data
drive.mount('/content/drive')

# defining the local path to google drive
images_path = f'/content/drive/MyDrive/images_original'


if not os.path.exists(images_path):
    
    images_path = f'/content/drive/MyDrive/images_original'

    if not os.path.exists(images_path):
        raise FileNotFoundError(f'Directory not found: {images_path}')



# Transforming the images, as per requirement
transform = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor(),
])

# Creating an Imagefolder dataset, to use throughout the project
dataset = datasets.ImageFolder(root=images_path, transform=transform)

# Creating a DataLoader for batching and shuffling the data
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# cross checking the number of classes present in dataset
num_classes = len(dataset.classes)
class_names = dataset.classes

print(f"Number of classes: {num_classes}")
print("Class names:", class_names)



## Splitting the dataset 

In [ ]:
from torch.utils.data import random_split

# Specifying the sizes for training, validation, and test sets
total_samples = len(dataset)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)
test_size = test_size = total_samples - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Creating DataLoaders for each split
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Fully connectional network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


# Defining the Fully Connected Network
class FullyConnectedNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FullyConnectedNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# setting the Hyperparameters
input_size = 180 * 180 * 3 
hidden_size = 128
num_classes = 10  
learning_rate = 0.001
batch_size = 32
num_epochs_50 = 50
num_epochs_100 = 100

# Creating the model, loss function, and optimizer
model_50 = FullyConnectedNetwork(input_size, hidden_size, num_classes)
model_100 = FullyConnectedNetwork(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer_50 = optim.Adam(model_50.parameters(), lr=learning_rate)
optimizer_100 = optim.Adam(model_100.parameters(), lr=learning_rate)



# Training the model for 50 epochs
for epoch in range(num_epochs_50):
    for inputs, labels in train_loader:
        outputs = model_50(inputs)
        loss = criterion(outputs, labels)

        optimizer_50.zero_grad()
        loss.backward()
        optimizer_50.step()

    print(f'Epoch [{epoch+1}/{num_epochs_50}], Loss: {loss.item():.4f}')


# Training the model for 100 epochs
for epoch in range(num_epochs_100):
    for inputs, labels in train_loader:
        outputs = model_100(inputs)
        loss = criterion(outputs, labels)

        optimizer_100.zero_grad()
        loss.backward()
        optimizer_100.step()
    print(f'Epoch [{epoch+1}/{num_epochs_100}], Loss: {loss.item():.4f}')


# Evaluation on validation set

model_50.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model_50(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_1 = correct / total
print(f'Validation Accuracy for 50 epochs: {accuracy_1 * 100:.2f}%')

model_100.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model_100(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_2 = correct / total
print(f'Validation Accuracy for 100 epochs: {accuracy_2 * 100:.2f}%')

## Convolutional neural network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

# Defining a Convolutional Neural Network
class ModifiedConvolutionalNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv1_1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.relu1_1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.conv2_1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2_1 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 45 * 45, 512)
        self.relu_fc = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu1_1(self.conv1_1(x))
        x = self.maxpool1(x)
        x = self.relu2(self.conv2(x))
        x = self.relu2_1(self.conv2_1(x))
        x = self.maxpool2(x)

        x = x.view(x.size(0), -1)
        x = self.relu_fc(self.fc1(x))
        x = self.fc2(x)
        return x

# defining the Hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 23
num_epochs = 50

# Creating the model, loss function, and optimizer
modified_model = ModifiedConvolutionalNetwork(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modified_model.parameters(), lr=learning_rate)

# Creating DataLoader for training
modified_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training the model
for epoch in range(num_epochs):
    for inputs, labels in modified_train_loader:
        outputs = modified_model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Final evaluation on the validation set
modified_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = modified_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_modified_model = correct / total
print(f'Validation Accuracy with Modified Model: {accuracy_modified_model * 100:.2f}%')

## Convolutional neural network with batch normalization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

# Define a Convolutional Neural Network with Batch Normalization
class ModifiedConvolutionalNetworkWithBatchNorm(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedConvolutionalNetworkWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.conv1_1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(32)
        self.relu1_1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.conv2_1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(64)
        self.relu2_1 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 45 * 45, 512)
        self.bn_fc = nn.BatchNorm1d(512)
        self.relu_fc = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu1_1(self.bn1_1(self.conv1_1(x)))
        x = self.maxpool1(x)
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu2_1(self.bn2_1(self.conv2_1(x)))
        x = self.maxpool2(x)

        x = x.view(x.size(0), -1)
        x = self.relu_fc(self.bn_fc(self.fc1(x)))
        x = self.fc2(x)
        return x

# defining Hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 23
num_epochs = 100

# Create the model, loss function, and optimizer
modified_model_with_bn = ModifiedConvolutionalNetworkWithBatchNorm(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modified_model_with_bn.parameters(), lr=learning_rate)

modified_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training the model
for epoch in range(num_epochs):
    for inputs, labels in modified_train_loader:
        outputs = modified_model_with_bn(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Final evaluation on the validation set
modified_model_with_bn.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = modified_model_with_bn(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_modified_model_with_bn = correct / total
print(f'Validation Accuracy with Modified Model and Batch Normalization: {accuracy_modified_model_with_bn * 100:.2f}%')

## Convolutional neural network with RMSprop optimizer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

# Define a modified Convolutional Neural Network with Batch Normalization
class ModifiedConvolutionalNetworkWithBatchNorm(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedConvolutionalNetworkWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.conv1_1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(32)
        self.relu1_1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.conv2_1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(64)
        self.relu2_1 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 45 * 45, 512)
        self.bn_fc = nn.BatchNorm1d(512)
        self.relu_fc = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu1_1(self.bn1_1(self.conv1_1(x)))
        x = self.maxpool1(x)
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu2_1(self.bn2_1(self.conv2_1(x)))
        x = self.maxpool2(x)

        x = x.view(x.size(0), -1)
        x = self.relu_fc(self.bn_fc(self.fc1(x)))
        x = self.fc2(x)
        return x

# Hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 23
num_epochs = 100

# Create the model, loss function, and optimizer (RMSprop)
modified_model_with_bn = ModifiedConvolutionalNetworkWithBatchNorm(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(modified_model_with_bn.parameters(), lr=learning_rate)

# Create DataLoader for training
modified_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training the model
for epoch in range(num_epochs):
    for inputs, labels in modified_train_loader:
        outputs = modified_model_with_bn(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Final evaluation on the validation set
modified_model_with_bn.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = modified_model_with_bn(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_modified_model_with_bn = correct / total
print(f'Validation Accuracy with Modified Model and Batch Normalization: {accuracy_modified_model_with_bn * 100:.2f}%')